# Camera Calibration Demo
This example uses the web cam to grab a set of images of a 
checkerboard.  The images are then processed to calibrate the web camera

Results (and images) are saved.



In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import time

num_images = 10
pause_time = 2

filename = os.path.relpath('../../CameraCalibration/Calibration');

In [2]:
source = cv2.VideoCapture(0)
has_frame, cur_frame = source.read()

[ WARN:0@0.750] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [3]:
alive = True

win_name = "Motion Correction"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
source = cv2.VideoCapture(0)


time.sleep(pause_time)

for idx in range(0,num_images):
    filename_local = filename + str(idx) + '.jpg'
    print(filename_local)
    has_frame, cur_frame = source.read()
    cur_frame = cv2.flip(cur_frame, 1)
    cur_frame = cv2.cvtColor(cur_frame,cv2.COLOR_BGR2GRAY)

    cv2.imshow(win_name,cur_frame)
    cv2.imwrite(filename_local,cur_frame)
    key = cv2.waitKey(1)
    time.sleep(pause_time)

cv2.destroyWindow(win_name)
source.release()

[ WARN:0@3.305] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


../../CameraCalibration/Calibration0.jpg


2024-10-14 18:35:09.032 python[27596:3144189] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (2.02 secs).


../../CameraCalibration/Calibration1.jpg
../../CameraCalibration/Calibration2.jpg
../../CameraCalibration/Calibration3.jpg
../../CameraCalibration/Calibration4.jpg
../../CameraCalibration/Calibration5.jpg
../../CameraCalibration/Calibration6.jpg
../../CameraCalibration/Calibration7.jpg
../../CameraCalibration/Calibration8.jpg
../../CameraCalibration/Calibration9.jpg


Now, load the data back in and run the calibration code

In [3]:
import glob

img_filenames = glob.glob(os.path.relpath(filename + "*.jpg"))
img_filenames=sorted(img_filenames)

img_list = [cv2.imread(file) for file in img_filenames]

img_list=[cv2.cvtColor(img_list[idx],cv2.COLOR_BGR2GRAY) for idx in range(0,len(img_list))]


In [4]:
img_corners=[]
obj_corners=[]

CHECKERBOARD = (8,6)  # The size is number of corners, 
# which is one less than the number of squares

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

for idx in range(0,len(img_list)):
    retval, corners = cv2.findChessboardCorners(img_list[idx],patternSize =CHECKERBOARD)
    if(retval):
        corners=cv2.cornerSubPix(img_list[idx], corners, (11,11),(-1,-1), criteria)
        img_corners.append(corners)
        obj_corners.append(objp)
        img_list[idx] = cv2.drawChessboardCorners(img_list[idx], CHECKERBOARD, corners, retval)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_corners, img_corners, img_list[0].shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("distortion parameters : \n")
print(dist)

# Save in NumPy Zip format
np.savez(filename +"_camera.npz",mtx=mtx,dist=dist)


Camera matrix : 

[[1.05113819e+03 0.00000000e+00 7.35663076e+02]
 [0.00000000e+00 1.03757353e+03 3.38242702e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[ 3.01137568e-02  5.28221992e-01  4.14826599e-04  4.26752845e-02
  -9.93206370e-01]]
rvecs : 

(array([[ 0.06830113],
       [-0.35170619],
       [ 0.17207331]]), array([[ 0.01909188],
       [ 0.34489394],
       [-0.09420104]]), array([[ 0.00364926],
       [ 0.24310462],
       [-0.04676415]]), array([[-0.00545151],
       [-0.09183089],
       [ 0.05819735]]), array([[-0.0209604 ],
       [-0.17700663],
       [ 0.06653545]]), array([[-0.05649507],
       [-0.63349374],
       [ 0.20310309]]), array([[ 0.01935875],
       [-0.15861669],
       [ 0.04534504]]), array([[-0.04981367],
       [-0.086174  ],
       [ 0.05887108]]), array([[ 0.02147382],
       [ 0.44408708],
       [-0.06901087]]), array([[-0.00687212],
       [ 0.47128241],
       [-0.0565995 ]]))
tvecs : 

(array([[-17.89920333],
       [ -2.72779